In [1]:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, Dependency, FeatureType
from eolearn.core import OverwritePermission
# We'll use Sentinel-2 imagery (Level-1C) provided through Sentinel Hub
# If you don't know what `Level 1C` means, don't worry. It doesn't matter.
from eolearn.io import S2L1CWCSInput 
from eolearn.core import LoadFromDisk, SaveToDisk

# cloud detection
from eolearn.mask import AddCloudMaskTask, get_s2_pixel_cloud_detector
from eolearn.mask import AddValidDataMaskTask

# filtering of scenes
from eolearn.features import SimpleFilterTask

# burning the vectorised polygon to raster
from eolearn.geometry import VectorToRaster

# The golden standard: numpy and matplotlib
import numpy as np

# import matplotlib TODO
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# For manipulating geo-spatial vector dataset (polygons of nominal water extent)
import geopandas as gpd

# Image manipulations
# Our water detector is going to be based on a simple threshold 
# of Normalised Difference Water Index (NDWI) grayscale image
from skimage.filters import threshold_otsu

# Loading polygon of nominal water extent
import shapely.wkt
from shapely.geometry import Polygon

# sentinelhub-py package
from sentinelhub import BBox, CRS

from skimage.filters import sobel
from skimage.morphology import disk
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
import geopandas as gpd
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import imageio
import io
import os
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib
from ast import literal_eval
from datetime import datetime
#matplotlib.use('Agg')

/Users/gokul/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
seq = []
b = [51,
 74,
 79,
 117,
 128,
 129,
 155,
 157,
 159,
 167,
 175,
 176,
 177,
 178,
 179,
 208,
 241,
 256,
 278]
for i in range(0,290):
    if i not in b:
        seq+= [i]

In [3]:
source = gpd.read_file("source.geojson")
source["water_level"] = ""
source["date_range"] = ""

In [4]:
def plot_rgb_w_water(eopatch, idx,filepath, i, name):
    plt.ioff()
    #print(eopatch.bbox.max_x,eopatch.bbox.min_x,eopatch.bbox.max_y,eopatch.bbox.min_y)
    ratio = np.abs(eopatch.bbox.max_x - eopatch.bbox.min_x) / np.abs(eopatch.bbox.max_y - eopatch.bbox.min_y)
    #print(ratio)
    fig, ax = plt.subplots(figsize=(ratio * 10, 10))
    ax.imshow(eopatch.data['TRUE-COLOR-S2-L1C'][idx])
    observed = closing(eopatch.mask['WATER_MASK'][idx,...,0], disk(1))
    nominal = sobel(eopatch.mask_timeless['NOMINAL_WATER'][...,0])
    observed = sobel(observed)
    nominal = np.ma.masked_where(nominal == False, nominal)
    observed = np.ma.masked_where(observed == False, observed)
    ax.set_title(eopatch.timestamp[idx].date())
    ax.imshow(nominal, cmap=plt.cm.Reds)
    ax.imshow(observed, cmap=plt.cm.Blues)
    ax.axis('off')
    plt.savefig("data/"+str(i)+"/images/"+str(name)+".png", format='png',transparent = True, bbox_inches = 'tight', pad_inches = 0)
    return("data/"+str(i)+"/images/"+str(name)+".png")

In [5]:
def generate_data(i):
    filepath = 'patches/patches_'+str(i)
    patch = EOPatch.load(filepath)
    if not os.path.exists("data/"+str(i)+"/images/"):
        os.makedirs("data/"+str(i)+"/images/")
    files = []
    for j in range(0,len(patch.timestamp)): 
        im = plot_rgb_w_water(patch, j,filepath, i, patch.timestamp[j])
        files.append(im)
    images = []
    for filename in files:
        images.append(imageio.imread(filename))
    imageio.mimsave("data/"+str(i)+"/images/"+'all_images.gif', images,duration=0.5)
    dates = np.asarray(patch.timestamp)
    date = []
    for j in range(0,len(dates)): date.append(dates[j].strftime("%Y, %M, %d, %H, %m, %S"))
    source["water_level"][i] = str(patch.scalar['WATER_LEVEL'][patch.scalar['COVERAGE'][...,0] < 1.0].tolist())
    source["date_range"][i] = str(date)

In [7]:
for i in seq[200:]:
    print(i)
    generate_data(i)

216


/Users/gokul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
/Users/gokul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/gokul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
242
243
244
245
246
247
248
249
250
251
252
253
254
255
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
279
280
281
282
283
284
285
286
287
288
289


In [9]:
for i in seq:
    print(i)
    patch = EOPatch.load('patches/patches_'+str(i))
    dates = np.asarray(patch.timestamp)
    date = []
    for j in range(0,len(dates)): date.append(dates[j].strftime("%Y, %m, %d, %H, %M, %S"))
    source["water_level"][i] = str(patch.scalar['WATER_LEVEL'][patch.scalar['COVERAGE'][...,0] < 1.0].tolist())
    source["date_range"][i] = str(date)

0
1


/Users/gokul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/gokul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
75
76
77
78
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
118
119
120
121
122
123
124
125
126
127
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
156
158
160
161
162
163
164
165
166
168
169
170
171
172
173
174
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
242
243
244
245
246
247
248
249
250
251
252
253
254
255
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
279
280
281
282
283
284
285
286
287
288
289


In [10]:
source.to_file("data.geojson", driver="GeoJSON")